# von Neumann analysis

In [45]:
import numpy as np

def flux(q, method):
    def fluxk(q, k):
        return q + .25*(1+k)*(np.roll(q,-1)-q) + .25*(1-k)*(q-np.roll(q,1))
    fluxmeth = {
        'upwind1' : lambda q: q,
        'center2' : lambda q: .5*(q+np.roll(q,-1)),
        'fromm' : lambda q: fluxk(q, 0.),
        'quick' : lambda q: fluxk(q, .5),
        'upwind3' : lambda q: fluxk(q, 1./3.),
        'center4' : lambda q: .5*(q+np.roll(q,-1)) - .125*(2./3.)*(np.roll(q,-2)-np.roll(q,-1)-q+np.roll(q,1))
    }
    if method not in fluxmeth.keys():
        KeyError(f"'{method}' unknown")
    return fluxmeth[method](q)

In [48]:
def error(kh, method):
    imax = 2 # stencil of 5 points
    cshift = np.exp(1j*kh) # complex shift to get +1 index
    q = np.array([cshift**(i-imax) for i in range(2*imax+1)]) # q[imax] = 1
    rhs = -(flux(q, method)-flux(q/cshift, method)) # rhs on imax index
    err = rhs[imax] + 1j*kh
    # print('q',q)
    # print(flux(q, method))
    # print(rhs)
    return abs(err.real)+1j*abs(err.imag)

# for meth in ['upwind1', 'center2', 'fromm', 'quick', 'upwind3', 'center4']:
#     print(meth, error(.1, meth))

In [ ]:
import matplotlib.pyplot as plt
fig, (axr, axi) = plt.subplots(1, 2, figsize=(10, 4))
kh = np.geomspace(5e-2, np.pi, 100)
for ax, ylab in zip([axr, axi], ['dissipation error', 'dispersion error']):
    ax.set_xlabel('$\lambda/h$') ; ax.set_ylabel(ylab)
    ax.grid()
    ax.set_ylim(1e-4, 1e1)
for method in ['upwind1', 'center2', 'fromm', 'quick', 'upwind3', 'center4']:
    err = np.array([error(ikh, method) for ikh in kh])
    axr.loglog(2*np.pi/kh, err.real, label=method)
    axi.loglog(2*np.pi/kh, err.imag, label=method)
axr.legend() ; axi.legend()
